In [1]:
from surprise import KNNBasic, BaselineOnly, KNNWithMeans,KNNWithZScore, KNNBaseline
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader
import numpy as np

In [2]:
np.random.seed(42)

In [3]:
import pandas as pd
df = pd.read_parquet("user_item_ratio_stacked.parquet")
df.columns = ["userID", "itemID", "rating"]

In [4]:
filtered_df = df.groupby('userID')['itemID'].count().reset_index().loc[lambda x: x['itemID'] >= 3]


In [5]:
df= df.loc[df.userID.isin(filtered_df.userID)]

In [6]:
from surprise import SVD, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline, CoClustering, BaselineOnly, NormalPredictor
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np

In [7]:
from surprise import KNNBasic, KNNWithZScore, KNNWithMeans, Dataset, Reader
from surprise.model_selection import GridSearchCV, train_test_split

# Đảm bảo bạn đã có dữ liệu và đã tạo đối tượng Reader để đọc dữ liệu.
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra.
trainset, testset = train_test_split(data, test_size=0.2)


In [8]:
%%time
param_grid = {
    'k': [10, 20, 30],  # Thử nghiệm với các giá trị k khác nhau
    'sim_options': {
        'name': ['cosine','msd', 'pearson'],
        # 'user_based': [True]
    }
}

# Tạo đối tượng GridSearchCV để tìm tham số tối ưu.
grid_search = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mae'], cv=3)
grid_search.fit(data)

# In ra tham số tối ưu và hiệu suất tốt nhất.
print("Best RMSE Score:", grid_search.best_score['rmse'])
print("Best Parameters:", grid_search.best_params['rmse'])


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd sim

In [9]:
%%time
best_k = grid_search.best_params['rmse']['k']
best_sim_options = grid_search.best_params['rmse']['sim_options']

# Tạo mô hình KNN dựa trên tham số tối ưu.
model = KNNBasic(k=best_k, sim_options=best_sim_options)

# Huấn luyện mô hình trên tập huấn luyện.
model.fit(trainset)

# Dự đoán và đánh giá trên tập kiểm tra.
predictions = model.test(testset)

from surprise import accuracy
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.1902
MAE:  0.1331
CPU times: user 59.6 s, sys: 3.76 s, total: 1min 3s
Wall time: 1min 4s


## với KNNwithmean

In [10]:
%%time
param_grid = {
    'k': [10, 20, 30],  # Thử nghiệm với các giá trị k khác nhau
    'sim_options': {
        'name': ['cosine','msd', 'pearson'],
        # 'user_based': [True]
    }
}

# Tạo đối tượng GridSearchCV để tìm tham số tối ưu.
grid_search = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=3)
grid_search.fit(data)

# In ra tham số tối ưu và hiệu suất tốt nhất.
print("Best RMSE Score:", grid_search.best_score['rmse'])
print("Best Parameters:", grid_search.best_params['rmse'])


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd sim

In [11]:
%%time
best_k = grid_search.best_params['rmse']['k']
best_sim_options = grid_search.best_params['rmse']['sim_options']

# Tạo mô hình KNN dựa trên tham số tối ưu.
model = KNNBasic(k=best_k, sim_options=best_sim_options)

# Huấn luyện mô hình trên tập huấn luyện.
model.fit(trainset)

# Dự đoán và đánh giá trên tập kiểm tra.
predictions = model.test(testset)

from surprise import accuracy
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.1902
MAE:  0.1331
CPU times: user 1min 7s, sys: 5.71 s, total: 1min 13s
Wall time: 1min 14s


## KNNwithZscore

In [12]:
%%time
param_grid = {
    'k': [10, 20, 30],  # Thử nghiệm với các giá trị k khác nhau
    'sim_options': {
        'name': ['cosine','msd', 'pearson'],
        # 'user_based': [True]
    }
}

# Tạo đối tượng GridSearchCV để tìm tham số tối ưu.
grid_search = GridSearchCV(KNNWithZScore, param_grid, measures=['rmse', 'mae'], cv=3)
grid_search.fit(data)

# In ra tham số tối ưu và hiệu suất tốt nhất.
print("Best RMSE Score:", grid_search.best_score['rmse'])
print("Best Parameters:", grid_search.best_params['rmse'])


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd sim

In [13]:
%%time
best_k = grid_search.best_params['rmse']['k']
best_sim_options = grid_search.best_params['rmse']['sim_options']

# Tạo mô hình KNN dựa trên tham số tối ưu.
model = KNNBasic(k=best_k, sim_options=best_sim_options)

# Huấn luyện mô hình trên tập huấn luyện.
model.fit(trainset)

# Dự đoán và đánh giá trên tập kiểm tra.
predictions = model.test(testset)

from surprise import accuracy
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.1902
MAE:  0.1331
CPU times: user 1min, sys: 3.73 s, total: 1min 4s
Wall time: 1min 5s


## SVD

In [14]:
from surprise import Reader, Dataset, SVD, SVDpp, NMF, KNNBaseline , accuracy

In [15]:
%%time
param_grid = {
    'n_factors': [50, 100, 200],        # Thử nghiệm với số lượng factors khác nhau
    'n_epochs': [10, 20, 30],          # Thử nghiệm với số lượng epochs khác nhau
    'lr_all': [0.002, 0.005, 0.01],    # Thử nghiệm với learning rate khác nhau
    'reg_all': [0.02, 0.04, 0.06]      # Thử nghiệm với regularization khác nhau
}

# Tạo đối tượng GridSearchCV để tìm tham số tối ưu.
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
grid_search.fit(data)

# In ra tham số tối ưu và hiệu suất tốt nhất.
print("Best RMSE Score:", grid_search.best_score['rmse'])
print("Best Parameters:", grid_search.best_params['rmse'])


Best RMSE Score: 0.2337883719392916
Best Parameters: {'n_factors': 50, 'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.04}
CPU times: user 4min 5s, sys: 2.6 s, total: 4min 8s
Wall time: 4min 15s


In [16]:
%%time
best_n_factors = grid_search.best_params['rmse']['n_factors']
best_n_epochs = grid_search.best_params['rmse']['n_epochs']
best_lr_all = grid_search.best_params['rmse']['lr_all']
best_reg_all = grid_search.best_params['rmse']['reg_all']

# Tạo mô hình SVD dựa trên tham số tối ưu.
model = SVD(n_factors=best_n_factors, n_epochs=best_n_epochs, lr_all=best_lr_all, reg_all=best_reg_all)

# Huấn luyện mô hình trên tập huấn luyện.
model.fit(trainset)

# Dự đoán và đánh giá trên tập kiểm tra.
predictions = model.test(testset)

from surprise import accuracy
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)


RMSE: 0.2343
MAE:  0.1997
CPU times: user 252 ms, sys: 153 ms, total: 405 ms
Wall time: 405 ms


## SVD++

In [17]:
%%time
param_grid = {
    'n_epochs': [10, 20, 30],        # Thử nghiệm với số lượng epochs khác nhau
    'lr_all': [0.002, 0.005, 0.01],  # Thử nghiệm với learning rate khác nhau
    'reg_all': [0.02, 0.04, 0.06]    # Thử nghiệm với regularization khác nhau
}

# Tạo đối tượng GridSearchCV để tìm tham số tối ưu.
grid_search = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)
grid_search.fit(data)

# In ra tham số tối ưu và hiệu suất tốt nhất.
print("Best RMSE Score:", grid_search.best_score['rmse'])
print("Best Parameters:", grid_search.best_params['rmse'])


Best RMSE Score: 0.225803493217056
Best Parameters: {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.02}
CPU times: user 1min 22s, sys: 656 ms, total: 1min 23s
Wall time: 1min 24s


In [18]:
%%time
best_n_epochs = grid_search.best_params['rmse']['n_epochs']
best_lr_all = grid_search.best_params['rmse']['lr_all']
best_reg_all = grid_search.best_params['rmse']['reg_all']

# Tạo mô hình SVD++ dựa trên tham số tối ưu.
model = SVDpp(n_epochs=best_n_epochs, lr_all=best_lr_all, reg_all=best_reg_all)

# Huấn luyện mô hình trên tập huấn luyện.
model.fit(trainset)

# Dự đoán và đánh giá trên tập kiểm tra.
predictions = model.test(testset)

from surprise import accuracy
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.2258
MAE:  0.1913
CPU times: user 438 ms, sys: 6.37 ms, total: 445 ms
Wall time: 450 ms


In [19]:
a

NameError: name 'a' is not defined